# Train/Test the True Relationship Model

This notebook is design to predict DG relationships on the entity level. Here we are taking the input from the Bi-LSTM model, prior probability notebook and the summary statistics notebook and combinging it into a single dataset. From there we train a Ridge LR model and an elastic net LR model to make the final prediction.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from collections import defaultdict
import csv
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tqdm
from scipy.stats import fisher_exact
import scipy
from sqlalchemy import and_
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import average_precision_score, precision_recall_curve, roc_curve, auc, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import seaborn as sns

In [2]:
candidate_df = pd.read_csv("data/disease_gene_summary_stats.csv")
prior_df = pd.read_csv("data/observation-prior.csv")

In [3]:
candidate_df.head(10)

,disease_id,disease_name,gene_id,gene_name,nlog10_p_value,co_odds_ratio,co_expected_sen_count,delta_lower_ci,lstm_avg_marginal,lstm_marginal_0_quantile,lstm_marginal_20_quantile,lstm_marginal_40_quantile,lstm_marginal_60_quantile,lstm_marginal_80_quantile,hetnet_labels
0,DOID:12849,autistic disorder,3569,IL6,2.072881e-04,0.617773,62.795531,-0.001252,0.747175,0.075081,0.650967,0.907099,0.913931,0.916503,1
1,DOID:3070,malignant glioma,567,B2M,3.358850e-09,0.192395,86.200979,-0.025416,0.130409,0.076953,0.078801,0.081089,0.082576,0.086210,-1
2,DOID:1324,lung cancer,6647,SOD1,-0.000000e+00,0.142479,194.551269,-0.044854,0.085042,0.074244,0.075985,0.080222,0.084748,0.089595,-1
3,DOID:3121,gallbladder cancer,2353,FOS,1.315072e+00,2.319352,2.589809,-0.000705,0.080139,0.078177,0.078499,0.079471,0.080457,0.081399,-1
4,DOID:10652,Alzheimer's disease,5520,PPP2R2A,1.116009e+00,2.601331,1.549830,-0.022814,0.082038,0.077898,0.078693,0.078702,0.082385,0.085898,-1
5,DOID:5041,esophageal cancer,319100,TAAR6,7.489680e+00,24.599058,0.301847,0.020276,0.084278,0.078433,0.081272,0.082851,0.084216,0.086745,-1
6,DOID:2531,hematologic cancer,6400,SEL1L,6.776874e-02,0.674023,6.909290,-0.102290,0.081141,0.077944,0.079681,0.080332,0.081041,0.081786,-1
7,DOID:1936,atherosclerosis,22846,VASH1,4.202512e-01,1.528825,1.291256,-0.020213,0.087098,0.087098,0.087098,0.087098,0.087098,0.087098,-1
8,DOID:2377,multiple sclerosis,10804,GJB6,1.581787e+00,4.934903,0.613036,-0.018946,0.079142,0.077703,0.078278,0.078854,0.079430,0.080006,-1
9,DOID:14227,azoospermia,83844,USP26,3.600000e+01,4841.654246,0.009900,0.538260,0.170588,0.076621,0.079998,0.082054,0.090462,0.151144,1


In [4]:
prior_df.head(10)

,disease_id,gene_id,disease_associates,gene_associates,prior_perm,prior_perm_stderr
0,DOID:2531,7124,550,47,0.870898,0.000558
1,DOID:2531,7157,550,46,0.867094,0.000565
2,DOID:2531,5743,550,40,0.830758,0.000624
3,DOID:2531,3569,550,39,0.823473,0.000634
4,DOID:2531,4318,550,37,0.809203,0.000654
5,DOID:2531,3586,550,35,0.793287,0.000674
6,DOID:2531,7422,550,34,0.783978,0.000685
7,DOID:2531,7040,550,33,0.777711,0.000692
8,DOID:2531,3553,550,31,0.758141,0.000712
9,DOID:2531,213,550,30,0.747171,0.000723


# Set up the Training and Testing Set

In [5]:
train_df = pd.read_csv("stratified_data/train_set.csv")
dev_df = pd.read_csv("stratified_data/dev_set.csv")
test_df = pd.read_csv("stratified_data/test_set.csv")

In [6]:
# Gather the summary stats for each candidate
training_set = pd.merge(candidate_df, train_df.query("pubmed==1")[["disease_id", "gene_id", "hetnet"]], 
                        how='inner', on=["disease_id", "gene_id"])
dev_set = pd.merge(candidate_df, dev_df.query("pubmed==1")[["disease_id", "gene_id", "hetnet"]],
                   how='inner', on=["disease_id", "gene_id"])
test_set = pd.merge(candidate_df, test_df.query("pubmed==1")[["disease_id", "gene_id", "hetnet"]], 
                    how='inner', on=["disease_id", "gene_id"])


# Drop the values that aren't found in pubmed. 
training_set = training_set.drop("hetnet_labels", axis=1)
dev_set = dev_set.drop("hetnet_labels", axis=1)
test_set = test_set.drop("hetnet_labels", axis=1)

training_set = training_set.dropna()
dev_set = dev_set.dropna()
test_set = test_set.dropna()

# Add the prior prob to the different sets 
training_set = pd.merge(training_set, prior_df[["disease_id", "gene_id", "prior_perm"]])
dev_set = pd.merge(dev_set, prior_df[["disease_id", "gene_id", "prior_perm"]])
test_set = pd.merge(test_set, prior_df[["disease_id", "gene_id", "prior_perm"]])

In [7]:
non_features = [
    "hetnet", "disease_id", "gene_id", 
    "gene_name", "disease_name",
    "pubmed", "lstm_marginal_0_quantile", 
    "lstm_marginal_20_quantile","lstm_marginal_40_quantile",
    "lstm_marginal_60_quantile", "lstm_marginal_80_quantile"
]

X = training_set[[col for col in training_set.columns if col not in non_features]]
Y = training_set["hetnet"]

X_dev = dev_set[[col for col in dev_set.columns if col not in non_features]]
Y_dev = dev_set["hetnet"]

X_test = test_set[[col for col in test_set.columns if col not in non_features]]
Y_test = test_set["hetnet"]

# Train the Machine Learning Algorithms

Here we use gridsearch to optimize both models using 10 fold cross validation. After exhausting the list of parameters, the best model is chosen and analyzed in the next chunk. 

In [8]:
n_iter = 100
final_models = []

lr = LogisticRegression()
lr_grid = {'C':np.linspace(1, 100, num=100)}

no_lstm_normalizer = StandardScaler()
lstm_normalizer = StandardScaler()

In [9]:
%%time

# Train on data without LSTM input
lstm_features = [
    "lstm_avg_marginal"
]

tempX = X[[col for col in X.columns if col not in lstm_features]]
tempX = tempX.append(X_dev[[col for col in X_dev.columns if col not in lstm_features]])

transformed_tempX = no_lstm_normalizer.fit_transform(tempX)
transformed_X = lstm_normalizer.fit_transform(X.append(X_dev))

CPU times: user 11.4 ms, sys: 0 ns, total: 11.4 ms
Wall time: 12 ms


In [ ]:
%%time

final_model = GridSearchCV(lr, lr_grid, cv=10, n_jobs=3, scoring='roc_auc', return_train_score=True)
final_model.fit(transformed_tempX, Y.append(Y_dev))
final_models.append(final_model)

CPU times: user 7.99 s, sys: 413 ms, total: 8.4 s
Wall time: 1min 42s


In [ ]:
%%time

# Train on data with LSTM input
final_model = GridSearchCV(lr, lr_grid, cv=10, n_jobs=3, scoring='roc_auc', return_train_score=True)
final_model.fit(transformed_X, Y.append(Y_dev))
final_models.append(final_model)

## Parameter Optimization

In [ ]:
no_lstm_result = pd.DataFrame(final_models[0].cv_results_)
lstm_result = pd.DataFrame(final_models[1].cv_results_)

In [ ]:
# No LSTM
plt.plot(no_lstm_result['param_C'], no_lstm_result['mean_test_score'])

In [ ]:
# LSTM
plt.plot(lstm_result['param_C'], lstm_result['mean_test_score'])

## LR Weights

In [ ]:
list(zip(final_models[0].best_estimator_.coef_[0], [col for col in training_set.columns if col not in lstm_features+non_features]))

In [ ]:
list(zip(final_models[1].best_estimator_.coef_[0], [col for col in training_set.columns if col not in non_features]))

# AUROCS

In [ ]:
feature_rocs = []
train_Y = Y.append(Y_dev)
train_X = X.append(X_dev)
for feature in X.columns:
    fpr, tpr, _ = roc_curve(train_Y, train_X[feature])
    feature_auc = auc(fpr, tpr)
    feature_rocs.append((feature, feature_auc))

feature_roc_df = pd.DataFrame(feature_rocs, columns=["Feature", "AUROC"])
ax = sns.barplot(x="AUROC", y="Feature", data=feature_roc_df)
plt.xlim([0.5,1])

# Corerlation Matrix

In [ ]:
feature_corr_mat = train_X.corr()
sns.heatmap(feature_corr_mat, cmap="RdBu", center=0)

# ML Performance

In [ ]:
transformed_tempX_test = no_lstm_normalizer.transform(X_test[[col for col in X.columns if col not in lstm_features]])
transformed_X_test = lstm_normalizer.transform(X_test)

In [ ]:
colors = ["green","red"]
labels = ["LR_NO_LSTM","LR_LSTM"]

In [ ]:
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label="Random")

# Plot the p_values log transformed
fpr, tpr, thresholds= roc_curve(Y_test, X_test["prior_perm"])
model_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, color='cyan', label="{} (area = {:0.2f})".format("prior", model_auc))

fpr, tpr, thresholds= roc_curve(Y_test, final_models[0].predict_proba(transformed_tempX_test)[:,1])
model_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, color=colors[0], label="{} (area = {:0.2f})".format(labels[0], model_auc))

fpr, tpr, thresholds= roc_curve(Y_test, final_models[1].predict_proba(transformed_X_test)[:,1])
model_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, color=colors[1], label="{} (area = {:0.2f})".format(labels[1], model_auc))

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
plt.legend(loc="lower right")

In [ ]:
plt.figure()

# Plot the p_values log transformed
precision, recall, _= precision_recall_curve(Y_test, X_test["prior_perm"])
model_precision = average_precision_score(Y_test, X_test["prior_perm"])
plt.plot(recall, precision, color='cyan', label="{} (area = {:0.2f})".format("prior", model_precision))

precision, recall, _ = precision_recall_curve(Y_test, final_models[0].predict_proba(transformed_tempX_test)[:,1])
model_precision = average_precision_score(Y_test, final_models[0].predict_proba(transformed_tempX_test)[:,1])
plt.plot(recall, precision, color=colors[0], label="{} curve (area = {:0.2f})".format(labels[0], model_precision))
  
precision, recall, _ = precision_recall_curve(Y_test, final_models[1].predict_proba(transformed_X_test)[:,1])
model_precision = average_precision_score(Y_test, final_models[1].predict_proba(transformed_X_test)[:,1])
plt.plot(recall, precision, color=colors[1], label="{} curve (area = {:0.2f})".format(labels[1], model_precision))

plt.ylabel('Precision')
plt.xlabel('Recall')
plt.title('Precision-Recall Curve')
plt.xlim([0, 1.01])
plt.ylim([0, 1.05])
plt.legend(loc="upper right")

## Save Final Result in DF

In [ ]:
predictions = final_models[1].predict_proba(X.append(X_dev).append(X_test))
predictions_df = training_set.append(dev_set).append(test_set)[[
    "disease_id","disease_name", 
    "gene_id", "gene_name", 'hetnet']]
predictions_df["predictions"] = predictions[:,1]

In [ ]:
predictions_df.to_csv("data/vanilla_lstm/final_model_predictions.csv", index=False)